In [38]:
import os
import sys
import ctypes
import math
import numpy
import numpy as np
import h5py
import tiledb

In [39]:
# Total number of time steps
# num_time_steps = 257
num_time_steps = 5

# Shape of the array for each time step
array_shape = (1024, 1024, 1024)

# Iterate over the .npy files and write to the HDF5 dataset
for timestep in range(1,num_time_steps):
    # Load the .npy file
    file_path = f'/home2/aoyagir/isotropic1024coarse/pressure2_time_steps/t_{timestep:04}.npy'
    if os.path.exists(file_path):
        # print("iine")
        pass
    else:
        print("doesnt exists")

In [40]:
# Set the TileDB context and configure the array schema
ctx = tiledb.Ctx()

array_name = f"array_{num_time_steps}"
# array_name = f"array_{num_time_steps}_lz4"
# array_name = f"array_{num_time_steps}_zstd"
# array_name = f"array_{num_time_steps}_gzip"


# array_uri = f"/home2/aoyagir/tiledb_data/{array_name}"
array_uri = f"/scratch/aoyagir/minio_data/tiledb/{array_name}"

array_schema = tiledb.ArraySchema(
    domain=tiledb.Domain(
        tiledb.Dim(name="timestep",domain=(0, num_time_steps - 1), tile=1, dtype=np.uint64),
        tiledb.Dim(name="x", domain=(0, 1023), tile=256, dtype=np.uint64),
        tiledb.Dim(name="y", domain=(0, 1023), tile=256, dtype=np.uint64),
        tiledb.Dim(name="z", domain=(0, 1023), tile=256, dtype=np.uint64),
    ),
    attrs=[tiledb.Attr(name="data", dtype=np.float32)],
    # filters=tiledb.FilterList([tiledb.LZ4Filter()]),  # Enable LZ4 compression
    # filters=tiledb.FilterList([gzip_filter])
    # filters=tiledb.FilterList([zstd_filter])
    # filters=tiledb.FilterList([blosc_filter])
)

if not tiledb.object_type(array_uri, ctx=ctx):
    tiledb.Array.create(array_uri, array_schema)

In [41]:
print(slice(0,1023))
a = (slice(0,1023))

slice(0, 1023, None)


In [42]:
# Iterate over the .npy files and write to the TileDB dataset
for timestep in range(1, num_time_steps):
    file_path = f'/home2/aoyagir/isotropic1024coarse/pressure2_time_steps/t_{timestep:04}.npy'
    if os.path.exists(file_path):
        # Load the .npy file
        data_3d = np.load(file_path).astype(np.float32)

        # Open the TileDB array for writing
        with tiledb.DenseArray(array_uri, mode="w", ctx=ctx) as array:
            # Define the coordinates for this timestep
            coords = (timestep-1, slice(0, 1024), slice(0, 1024), slice(0, 1024))
            # Write the 3D data to the TileDB array
            array[coords] = data_3d
        print(f"timestep{timestep} is done")
    else:
        print(f"File doesn't exist for timestep {timestep}")


timestep1 is done
timestep2 is done
timestep3 is done
timestep4 is done


In [43]:
# read the file
with tiledb.DenseArray(array_uri, mode="r", ctx=ctx) as array:
    # Read data for a specific timestep, x, y, z
    timestep = 1
    x = 1000
    y = 1000
    z = 1000
    data = array[timestep, x, y, z]
    print(data)

OrderedDict([('data', array(0.3150467, dtype=float32))])


In [44]:
# slice the data 
with tiledb.DenseArray(array_uri, mode="r", ctx=ctx) as array:
    # Read the sliced data from the TileDB array using 'from' and 'to' expressions
    data = array[1, 0:256, 0:256, 0:256]
    print(data)


OrderedDict([('data', array([[[ 0.3053694 ,  0.2976655 ,  0.28795236, ..., -0.28638342,
         -0.29360807, -0.30097735],
        [ 0.30436593,  0.2952228 ,  0.2838828 , ..., -0.29048863,
         -0.2972544 , -0.30433214],
        [ 0.296399  ,  0.2840166 ,  0.27213436, ..., -0.2952025 ,
         -0.3017412 , -0.30872965],
        ...,
        [-0.78725076, -0.78911626, -0.7908375 , ...,  0.02542291,
          0.02436218,  0.02173092],
        [-0.782719  , -0.784177  , -0.78539985, ...,  0.02399603,
          0.02203757,  0.02003286],
        [-0.777802  , -0.77897257, -0.77992076, ...,  0.0241271 ,
          0.02226389,  0.02131218]],

       [[ 0.30238938,  0.29829192,  0.29373628, ..., -0.2813281 ,
         -0.2888389 , -0.2969703 ],
        [ 0.30204046,  0.29705822,  0.29095107, ..., -0.28609055,
         -0.2930829 , -0.3008195 ],
        [ 0.29479116,  0.2878434 ,  0.28100026, ..., -0.29144186,
         -0.29812127, -0.30557385],
        ...,
        [-0.8048645 , -0.8068158